In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("middleearth")

create_database('blp_dataset')
create_table('products_me')

In [ ]:
original_images = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\product")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for folder_up in os.listdir(folder):
            path_folder_up = Path(f"{folder}\{folder_up}")

            for file in os.listdir(path_folder_up):
                page_content = BeautifulSoup(open(Path(f"{path_folder_up}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                desc_content = page_content.findAll("div", attrs = {"class": "description"})

                if desc_content is not None and len(desc_content) > 1:
                    raw_desc = desc_content[1]

                    if raw_desc is not None:
                        description = get_content(raw_desc)

                ## GET TITLE
                title = ""
                raw_title = page_content.find("div", attrs = {"class": "product-header-left"})
                
                if raw_title is not None:
                    content_title = raw_title.find("h1")

                    if content_title is not None:
                        title = get_content(content_title)

                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_me (name, description, market_name, illegal) VALUES (%s, %s, %s, %s)"
                    values = (title, description, "Middle Earth", "t")

                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid

                    ## GET IMAGE
                    img_content = page_content.find("div", attrs = {"class": "image"})
                    img_name = ""

                    if img_content is not None:
                        img_a = img_content.find("a")
                        
                        if img_a is not None:
                            img_name = img_a["href"]
                    
                            if img_name is not "":
                                url_image = Path(f"{folder_image}\{img_name}")

                                # copy images to new directory, renaming with product id
                                if url_image.exists():
                                    print(f"Image {product_id} exists... creating")

                                    dst_dir = Path(f"D:\images\_middleearth\{product_id}.jpg")
                                    copyfile(url_image, dst_dir)
                                else:
                                    print(f"Image {product_id} does not exists")